# <b><span style='color:#F1A424'>AutoML - Binary Classification - Breast Cancer Prediction - Load Models </span> </b>

### Disclaimer
The sample code (“Sample Code”) provided is not covered by any Teradata agreements. Please be aware that Teradata has no control over the model responses to such sample code and such response may vary. The use of the model by Teradata is strictly for demonstration purposes and does not constitute any form of certification or endorsement. The sample code is provided “AS IS” and any express or implied warranties, including the implied warranties of merchantability and fitness for a particular purpose, are disclaimed. In no event shall Teradata be liable for any direct, indirect, incidental, special, exemplary, or consequential damages (including, but not limited to, procurement of substitute goods or services; loss of use, data, or profits; or business interruption) sustained by you or a third party, however caused and on any theory of liability, whether in contract, strict liability, or tort arising in any way out of the use of this sample code, even if advised of the possibility of such damage.

## <b> Problem overview:</b>
    

**Dataset used - Breast Cancer Dataset**

**Features**:

- `ID number`
-  `Diagnosis` : (M = malignant, B = benign)

- Ten real-valued features are computed for each cell nucleus:
    - radius (mean of distances from center to points on the perimeter)
    - texture (standard deviation of gray-scale values)
    - perimeter
    - area
    - smoothness (local variation in radius lengths)
    - compactness (perimeter^2 / area - 1.0)
    - concavity (severity of concave portions of the contour)
    - concave points (number of concave portions of the contour)
    - symmetry
    - fractal dimension ("coastline approximation" - 1)

- The mean, standard error and "worst" or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, field 3 is Mean Radius, field 13 is Radius SE, field 23 is Worst Radius.

- All feature values are recoded with four significant digits.

- Missing attribute values: none


**Target Variable**:

- `diagnosis`: 357 benign, 212 malignant

        
**Objective**:

The primary objective is typically to build a model that can accurately predict the type of cancer i.e., benign and malignant.

**Usecase**:

Here, we will use AutoML(Automated Machine Learning) functionality to automate the entire process of developing a predictive model. It will perform feature exploration, feature engineering, data preparation, model training and evaluation on dataset in auto run and at end we will get leaderboard containined different models along with their performance. Model will also have rank associated with them which indicates which is best performing model for given data followed by other models.



In [1]:
# Importing AutoML from teradataml
from teradataml import AutoML, AutoClassifier

In [2]:
# Importing other important libraries
import getpass
from teradataml import create_context, remove_context
from teradataml import DataFrame
from teradataml import load_example_data
from teradataml import TrainTestSplit

In [3]:
# Create the connection.
host = getpass.getpass("Host: ")
username = getpass.getpass("Username: ")
password = getpass.getpass("Password: ")

con = create_context(host=host, username=username, password=password)

Host:  ········
Username:  ········
Password:  ········


## <b><span style='color:#F1A424'>| 1.</span> Loading Dataset </b>

In [4]:
# Loading dataset from example data collection
load_example_data('teradataml','breast_cancer')

In [5]:
# Fetching in teradata dataframe
cancer_df = DataFrame("breast_cancer")

In [6]:
# first 5 rows
cancer_df.head()

id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
879830,M,17.01,20.26,109.7,904.3,0.08772,0.07304,0.0695,0.0539,0.2026,0.05223,0.5858,0.8554,4.106,68.46,0.005038,0.01503,0.01946,0.01123,0.02294,0.002581,19.8,25.05,130.0,1210.0,0.1111,0.1486,0.1932,0.1096,0.3275,0.06469
894329,B,9.042,18.9,60.07,244.5,0.09968,0.1972,0.1975,0.04908,0.233,0.08743,0.4653,1.911,3.769,24.2,0.009845,0.0659,0.1027,0.02527,0.03491,0.007877,10.06,23.4,68.62,297.1,0.1221,0.3748,0.4609,0.1145,0.3135,0.1055
87930,B,12.47,18.6,81.09,481.9,0.09965,0.1058,0.08005,0.03821,0.1925,0.06373,0.3961,1.044,2.497,30.29,0.006953,0.01911,0.02701,0.01037,0.01782,0.003586,14.97,24.64,96.05,677.9,0.1426,0.2378,0.2671,0.1015,0.3014,0.0875
901303,B,16.17,16.07,106.3,788.5,0.0988,0.1438,0.06651,0.05397,0.199,0.06572,0.1745,0.489,1.349,14.91,0.00451,0.01812,0.01951,0.01196,0.01934,0.003696,16.97,19.14,113.1,861.5,0.1235,0.255,0.2114,0.1251,0.3153,0.0896
877159,M,18.08,21.84,117.4,1024.0,0.07371,0.08642,0.1103,0.05778,0.177,0.0534,0.6362,1.305,4.312,76.36,0.00553,0.05296,0.0611,0.01444,0.0214,0.005036,19.76,24.7,129.1,1228.0,0.08822,0.1963,0.2535,0.09181,0.2369,0.06558
855625,M,19.07,24.81,128.3,1104.0,0.09081,0.219,0.2107,0.09961,0.231,0.06343,0.9811,1.666,8.83,104.9,0.006548,0.1006,0.09723,0.02638,0.05333,0.007646,24.09,33.17,177.4,1651.0,0.1247,0.7444,0.7242,0.2493,0.467,0.1038
91544002,B,11.06,17.12,71.25,366.5,0.1194,0.1071,0.04063,0.04268,0.1954,0.07976,0.1779,1.03,1.318,12.3,0.01262,0.02348,0.018,0.01285,0.0222,0.008313,11.69,20.74,76.08,411.1,0.1662,0.2031,0.1256,0.09514,0.278,0.1168
86973701,B,14.95,18.77,97.84,689.5,0.08138,0.1167,0.0905,0.03562,0.1744,0.06493,0.422,1.909,3.271,39.43,0.00579,0.04877,0.05303,0.01527,0.03356,0.009368,16.25,25.47,107.1,809.7,0.0997,0.2521,0.25,0.08405,0.2852,0.09218
902975,B,12.21,14.09,78.78,462.0,0.08108,0.07823,0.06839,0.02534,0.1646,0.06154,0.2666,0.8309,2.097,19.96,0.004405,0.03026,0.04344,0.01087,0.01921,0.004622,13.13,19.29,87.65,529.9,0.1026,0.2431,0.3076,0.0914,0.2677,0.08824
914862,B,15.04,16.74,98.73,689.4,0.09883,0.1364,0.07721,0.06142,0.1668,0.06869,0.372,0.8423,2.304,34.84,0.004123,0.01819,0.01996,0.01004,0.01055,0.003237,16.76,20.43,109.7,856.9,0.1135,0.2176,0.1856,0.1018,0.2177,0.08549


In [7]:
# cancer_df dataset
cancer_df.shape

(569, 32)

In [8]:
# Performing sampling to get 80% for trainning and 20% for testing
cancer_df_sample = cancer_df.sample(frac = [0.8, 0.2])

In [9]:
cancer_df_sample.head()

id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst,sampleid
8913049,B,11.26,19.96,73.72,394.1,0.0802,0.1181,0.09274,0.05588,0.2595,0.06233,0.4866,1.905,2.877,34.68,0.01574,0.08262,0.08099,0.03487,0.03418,0.006517,11.86,22.33,78.27,437.6,0.1028,0.1843,0.1546,0.09314,0.2955,0.07009,1
84799002,M,14.54,27.54,96.73,658.8,0.1139,0.1595,0.1639,0.07364,0.2303,0.07077,0.37,1.033,2.879,32.55,0.005607,0.0424,0.04741,0.0109,0.01857,0.005466,17.46,37.13,124.1,943.2,0.1678,0.6577,0.7026,0.1712,0.4218,0.1341,1
8860702,M,17.3,17.08,113.0,928.2,0.1008,0.1041,0.1266,0.08353,0.1813,0.05613,0.3093,0.8568,2.193,33.63,0.004757,0.01503,0.02332,0.01262,0.01394,0.002362,19.85,25.09,130.9,1222.0,0.1416,0.2405,0.3378,0.1857,0.3138,0.08113,1
90602302,M,15.5,21.08,102.9,803.1,0.112,0.1571,0.1522,0.08481,0.2085,0.06864,1.37,1.213,9.424,176.5,0.008198,0.03889,0.04493,0.02139,0.02018,0.005815,23.17,27.65,157.1,1748.0,0.1517,0.4002,0.4211,0.2134,0.3003,0.1048,1
877159,M,18.08,21.84,117.4,1024.0,0.07371,0.08642,0.1103,0.05778,0.177,0.0534,0.6362,1.305,4.312,76.36,0.00553,0.05296,0.0611,0.01444,0.0214,0.005036,19.76,24.7,129.1,1228.0,0.08822,0.1963,0.2535,0.09181,0.2369,0.06558,2
86973701,B,14.95,18.77,97.84,689.5,0.08138,0.1167,0.0905,0.03562,0.1744,0.06493,0.422,1.909,3.271,39.43,0.00579,0.04877,0.05303,0.01527,0.03356,0.009368,16.25,25.47,107.1,809.7,0.0997,0.2521,0.25,0.08405,0.2852,0.09218,1
8610637,M,18.05,16.15,120.2,1006.0,0.1065,0.2146,0.1684,0.108,0.2152,0.06673,0.9806,0.5505,6.311,134.8,0.00794,0.05839,0.04658,0.0207,0.02591,0.007054,22.39,18.91,150.1,1610.0,0.1478,0.5634,0.3786,0.2102,0.3751,0.1108,1
927241,M,20.6,29.33,140.1,1265.0,0.1178,0.277,0.3514,0.152,0.2397,0.07016,0.726,1.595,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.74,39.42,184.6,1821.0,0.165,0.8681,0.9387,0.265,0.4087,0.124,1
904647,B,11.93,10.91,76.14,442.7,0.08872,0.05242,0.02606,0.01796,0.1601,0.05541,0.2522,1.045,1.649,18.95,0.006175,0.01204,0.01376,0.005832,0.01096,0.001857,13.8,20.14,87.64,589.5,0.1374,0.1575,0.1514,0.06876,0.246,0.07262,1
894329,B,9.042,18.9,60.07,244.5,0.09968,0.1972,0.1975,0.04908,0.233,0.08743,0.4653,1.911,3.769,24.2,0.009845,0.0659,0.1027,0.02527,0.03491,0.007877,10.06,23.4,68.62,297.1,0.1221,0.3748,0.4609,0.1145,0.3135,0.1055,1


In [10]:
# Fetching train and test data
cancer_df_train= cancer_df_sample[cancer_df_sample['sampleid'] == 1].drop('sampleid', axis=1)
cancer_df_test = cancer_df_sample[cancer_df_sample['sampleid'] == 2].drop('sampleid', axis=1)

In [11]:
# train data shape
cancer_df_train.shape

(455, 32)

In [12]:
# test data shape
cancer_df_test.shape

(114, 32)

In [13]:
#train dataset
cancer_df_train.head()

id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
8610637,M,18.05,16.15,120.2,1006.0,0.1065,0.2146,0.1684,0.108,0.2152,0.06673,0.9806,0.5505,6.311,134.8,0.00794,0.05839,0.04658,0.0207,0.02591,0.007054,22.39,18.91,150.1,1610.0,0.1478,0.5634,0.3786,0.2102,0.3751,0.1108
895633,M,16.26,21.88,107.5,826.8,0.1165,0.1283,0.1799,0.07981,0.1869,0.06532,0.5706,1.457,2.961,57.72,0.01056,0.03756,0.05839,0.01186,0.04022,0.006187,17.73,25.21,113.7,975.2,0.1426,0.2116,0.3344,0.1047,0.2736,0.07953
869104,M,16.11,18.05,105.1,813.0,0.09721,0.1137,0.09447,0.05943,0.1861,0.06248,0.7049,1.332,4.533,74.08,0.00677,0.01938,0.03067,0.01167,0.01875,0.003434,19.92,25.27,129.0,1233.0,0.1314,0.2236,0.2802,0.1216,0.2792,0.08158
852973,M,15.3,25.27,102.4,732.4,0.1082,0.1697,0.1683,0.08751,0.1926,0.0654,0.439,1.012,3.498,43.5,0.005233,0.03057,0.03576,0.01083,0.01768,0.002967,20.27,36.71,149.3,1269.0,0.1641,0.611,0.6335,0.2024,0.4027,0.09876
879830,M,17.01,20.26,109.7,904.3,0.08772,0.07304,0.0695,0.0539,0.2026,0.05223,0.5858,0.8554,4.106,68.46,0.005038,0.01503,0.01946,0.01123,0.02294,0.002581,19.8,25.05,130.0,1210.0,0.1111,0.1486,0.1932,0.1096,0.3275,0.06469
86973701,B,14.95,18.77,97.84,689.5,0.08138,0.1167,0.0905,0.03562,0.1744,0.06493,0.422,1.909,3.271,39.43,0.00579,0.04877,0.05303,0.01527,0.03356,0.009368,16.25,25.47,107.1,809.7,0.0997,0.2521,0.25,0.08405,0.2852,0.09218
914862,B,15.04,16.74,98.73,689.4,0.09883,0.1364,0.07721,0.06142,0.1668,0.06869,0.372,0.8423,2.304,34.84,0.004123,0.01819,0.01996,0.01004,0.01055,0.003237,16.76,20.43,109.7,856.9,0.1135,0.2176,0.1856,0.1018,0.2177,0.08549
855625,M,19.07,24.81,128.3,1104.0,0.09081,0.219,0.2107,0.09961,0.231,0.06343,0.9811,1.666,8.83,104.9,0.006548,0.1006,0.09723,0.02638,0.05333,0.007646,24.09,33.17,177.4,1651.0,0.1247,0.7444,0.7242,0.2493,0.467,0.1038
894329,B,9.042,18.9,60.07,244.5,0.09968,0.1972,0.1975,0.04908,0.233,0.08743,0.4653,1.911,3.769,24.2,0.009845,0.0659,0.1027,0.02527,0.03491,0.007877,10.06,23.4,68.62,297.1,0.1221,0.3748,0.4609,0.1145,0.3135,0.1055
91544002,B,11.06,17.12,71.25,366.5,0.1194,0.1071,0.04063,0.04268,0.1954,0.07976,0.1779,1.03,1.318,12.3,0.01262,0.02348,0.018,0.01285,0.0222,0.008313,11.69,20.74,76.08,411.1,0.1662,0.2031,0.1256,0.09514,0.278,0.1168


In [14]:
# test dataset
cancer_df_test.head()

id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
911150,B,14.53,19.34,94.25,659.7,0.08388,0.078,0.08817,0.02925,0.1473,0.05746,0.2535,1.354,1.994,23.04,0.004147,0.02048,0.03379,0.008848,0.01394,0.002327,16.3,28.39,108.1,830.5,0.1089,0.2649,0.3779,0.09594,0.2471,0.07463
869691,M,11.8,16.58,78.99,432.0,0.1091,0.17,0.1659,0.07415,0.2678,0.07371,0.3197,1.426,2.281,24.72,0.005427,0.03633,0.04649,0.01843,0.05628,0.004635,13.74,26.38,91.93,591.7,0.1385,0.4092,0.4504,0.1865,0.5774,0.103
872608,B,9.904,18.06,64.6,302.4,0.09699,0.1294,0.1307,0.03716,0.1669,0.08116,0.4311,2.261,3.132,27.48,0.01286,0.08808,0.1197,0.0246,0.0388,0.01792,11.26,24.39,73.07,390.2,0.1301,0.295,0.3486,0.0991,0.2614,0.1162
915186,B,9.268,12.87,61.49,248.7,0.1634,0.2239,0.0973,0.05252,0.2378,0.09502,0.4076,1.093,3.014,20.04,0.009783,0.04542,0.03483,0.02188,0.02542,0.01045,10.28,16.38,69.05,300.2,0.1902,0.3441,0.2099,0.1025,0.3038,0.1252
879523,M,15.12,16.68,98.78,716.6,0.08876,0.09588,0.0755,0.04079,0.1594,0.05986,0.2711,0.3621,1.974,26.44,0.005472,0.01919,0.02039,0.00826,0.01523,0.002881,17.77,20.24,117.7,989.5,0.1491,0.3331,0.3327,0.1252,0.3415,0.0974
902975,B,12.21,14.09,78.78,462.0,0.08108,0.07823,0.06839,0.02534,0.1646,0.06154,0.2666,0.8309,2.097,19.96,0.004405,0.03026,0.04344,0.01087,0.01921,0.004622,13.13,19.29,87.65,529.9,0.1026,0.2431,0.3076,0.0914,0.2677,0.08824
8911230,B,11.33,14.16,71.79,396.6,0.09379,0.03872,0.001487,0.003333,0.1954,0.05821,0.2375,1.28,1.565,17.09,0.008426,0.008998,0.001487,0.003333,0.02358,0.001627,12.2,18.99,77.37,458.0,0.1259,0.07348,0.004955,0.01111,0.2758,0.06386
877159,M,18.08,21.84,117.4,1024.0,0.07371,0.08642,0.1103,0.05778,0.177,0.0534,0.6362,1.305,4.312,76.36,0.00553,0.05296,0.0611,0.01444,0.0214,0.005036,19.76,24.7,129.1,1228.0,0.08822,0.1963,0.2535,0.09181,0.2369,0.06558
857438,M,15.1,22.02,97.26,712.8,0.09056,0.07081,0.05253,0.03334,0.1616,0.05684,0.3105,0.8339,2.097,29.91,0.004675,0.0103,0.01603,0.009222,0.01095,0.001629,18.1,31.69,117.7,1030.0,0.1389,0.2057,0.2712,0.153,0.2675,0.07873
89263202,M,20.09,23.86,134.7,1247.0,0.108,0.1838,0.2283,0.128,0.2249,0.07469,1.072,1.743,7.804,130.8,0.007964,0.04732,0.07649,0.01936,0.02736,0.005928,23.68,29.43,158.8,1696.0,0.1347,0.3391,0.4932,0.1923,0.3294,0.09469


## <b><span style='color:#F1A424'>| 2.</span> Loading Deployed Models - 'cancer_models_top_3' </b>

In [15]:
# Creating AutoML object

aml=AutoClassifier()

In [16]:
# Loading models

loaded_models = aml.load('cancer_models_top_3')

In [17]:
loaded_models

,RANK,MODEL_ID,FEATURE_SELECTION,ACCURACY,MICRO-PRECISION,MICRO-RECALL,MICRO-F1,MACRO-PRECISION,MACRO-RECALL,MACRO-F1,WEIGHTED-PRECISION,WEIGHTED-RECALL,WEIGHTED-F1,DATA_TABLE
0,1,SVM_1,rfe,0.978723,0.978723,0.978723,0.978723,0.978685,0.978685,0.978685,0.978723,0.978723,0.978723,ml__diagnosis_rfe_1768969051623974
1,2,SVM_2,pca,0.978723,0.978723,0.978723,0.978723,0.980392,0.977778,0.978636,0.979558,0.978723,0.978694,ml__diagnosis_pca_1768976549028885
2,3,SVM_0,lasso,0.872340,0.872340,0.872340,0.872340,0.901639,0.866667,0.868531,0.897454,0.872340,0.869484,ml__diagnosis_lasso_1768968394798123


## <b><span style='color:#F1A424'>| 3.</span> Get Hyperparameter for Loaded Model : </b>

In [19]:
aml.model_hyperparameters(rank=1)

{'response_column': 'diagnosis',
 'model_type': 'Classification',
 'lambda1': 0.1,
 'alpha': 0.85,
 'tolerance': 0.001,
 'learning_rate': 'OPTIMAL',
 'initial_eta': 0.05,
 'momentum': 0.95,
 'nesterov': True,
 'intercept': True,
 'iter_num_no_change': 10,
 'local_sgd_iterations ': 20,
 'iter_max': 200,
 'batch_size': 100,
 'output_prob': True,
 'output_responses': ['1', '0']}

In [21]:
aml.model_hyperparameters(rank=3)

{'response_column': 'diagnosis',
 'model_type': 'Classification',
 'lambda1': 0.001,
 'alpha': 0.15,
 'tolerance': 0.01,
 'learning_rate': 'OPTIMAL',
 'initial_eta': 0.1,
 'momentum': 0.65,
 'nesterov': True,
 'intercept': True,
 'iter_num_no_change': 5,
 'local_sgd_iterations ': 20,
 'iter_max': 200,
 'batch_size': 100,
 'output_prob': True,
 'output_responses': ['1', '0']}

## <b><span style='color:#F1A424'>| 4.</span> Generate Prediction using loaded models: </b>

In [22]:
# Generating prediction on test data
prediction = aml.predict(cancer_df_test, 2)

In [23]:
prediction

id,prediction,prob_0,prob_1,diagnosis
902975,0,1.0,0.0,0
879523,1,0.020324500953776914,0.9796754990462231,1
915186,0,0.999999111097718,8.889022820088821e-07,0
857438,1,0.013733713917352341,0.9862662860826477,1
869691,1,7.292784504153715e-05,0.9999270721549585,1
8911230,0,1.0,0.0,0
924934,0,1.0,0.0,0
89742801,1,0.0,1.0,1
872608,0,0.999999928476903,7.152309696264126e-08,0
877159,1,0.0,1.0,1


## <b><span style='color:#F1A424'>| 5.</span> Generate Prediction with preserve_columns: </b>

In [24]:
prediction_2=aml.predict(cancer_df_test, rank=3, preserve_columns=True)

In [25]:
prediction_2

id,prediction,prob_0,prob_1,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
8911230,1,0.0,1.0,B,11.33,14.16,71.79,396.6,0.09379,0.03872,0.001487,0.003333,0.1954,0.05821,0.2375,1.28,1.565,17.09,0.008426,0.008998,0.001487,0.003333,0.02358,0.001627,12.2,18.99,77.37,458.0,0.1259,0.07348,0.004955,0.01111,0.2758,0.06386
924934,1,0.0,1.0,B,10.29,27.61,65.67,321.4,0.0903,0.07658,0.05999,0.02738,0.1593,0.06127,0.2199,2.239,1.437,14.46,0.01205,0.02736,0.04804,0.01721,0.01843,0.004938,10.84,34.91,69.57,357.6,0.1384,0.171,0.2,0.09127,0.2226,0.08283
89263202,1,0.0,1.0,M,20.09,23.86,134.7,1247.0,0.108,0.1838,0.2283,0.128,0.2249,0.07469,1.072,1.743,7.804,130.8,0.007964,0.04732,0.07649,0.01936,0.02736,0.005928,23.68,29.43,158.8,1696.0,0.1347,0.3391,0.4932,0.1923,0.3294,0.09469
902975,1,0.0,1.0,B,12.21,14.09,78.78,462.0,0.08108,0.07823,0.06839,0.02534,0.1646,0.06154,0.2666,0.8309,2.097,19.96,0.004405,0.03026,0.04344,0.01087,0.01921,0.004622,13.13,19.29,87.65,529.9,0.1026,0.2431,0.3076,0.0914,0.2677,0.08824
911150,1,0.0,1.0,B,14.53,19.34,94.25,659.7,0.08388,0.078,0.08817,0.02925,0.1473,0.05746,0.2535,1.354,1.994,23.04,0.004147,0.02048,0.03379,0.008848,0.01394,0.002327,16.3,28.39,108.1,830.5,0.1089,0.2649,0.3779,0.09594,0.2471,0.07463
877159,1,0.0,1.0,M,18.08,21.84,117.4,1024.0,0.07371,0.08642,0.1103,0.05778,0.177,0.0534,0.6362,1.305,4.312,76.36,0.00553,0.05296,0.0611,0.01444,0.0214,0.005036,19.76,24.7,129.1,1228.0,0.08822,0.1963,0.2535,0.09181,0.2369,0.06558
879523,1,0.0,1.0,M,15.12,16.68,98.78,716.6,0.08876,0.09588,0.0755,0.04079,0.1594,0.05986,0.2711,0.3621,1.974,26.44,0.005472,0.01919,0.02039,0.00826,0.01523,0.002881,17.77,20.24,117.7,989.5,0.1491,0.3331,0.3327,0.1252,0.3415,0.0974
869691,1,0.0,1.0,M,11.8,16.58,78.99,432.0,0.1091,0.17,0.1659,0.07415,0.2678,0.07371,0.3197,1.426,2.281,24.72,0.005427,0.03633,0.04649,0.01843,0.05628,0.004635,13.74,26.38,91.93,591.7,0.1385,0.4092,0.4504,0.1865,0.5774,0.103
857438,1,0.0,1.0,M,15.1,22.02,97.26,712.8,0.09056,0.07081,0.05253,0.03334,0.1616,0.05684,0.3105,0.8339,2.097,29.91,0.004675,0.0103,0.01603,0.009222,0.01095,0.001629,18.1,31.69,117.7,1030.0,0.1389,0.2057,0.2712,0.153,0.2675,0.07873
89742801,1,0.0,1.0,M,17.06,21.0,111.8,918.6,0.1119,0.1056,0.1508,0.09934,0.1727,0.06071,0.8161,2.129,6.076,87.17,0.006455,0.01797,0.04502,0.01744,0.01829,0.003733,20.99,33.15,143.2,1362.0,0.1449,0.2053,0.392,0.1827,0.2623,0.07599


## <b><span style='color:#F1A424'>| 6.</span> Generate Evaluation Metrics

In [26]:
performance_metrics=aml.evaluate(cancer_df_test)

In [27]:
performance_metrics


############ output_data Output ############

   SeqNum              Metric  MetricValue
0       3        Micro-Recall     0.938596
1       5     Macro-Precision     0.946119
2       6        Macro-Recall     0.929924
3       7            Macro-F1     0.935991
4       9     Weighted-Recall     0.938596
5      10         Weighted-F1     0.938030
6       8  Weighted-Precision     0.941283
7       4            Micro-F1     0.938596
8       2     Micro-Precision     0.938596
9       1            Accuracy     0.938596


############ result Output ############

       Prediction  Mapping  CLASS_1  CLASS_2  Precision    Recall        F1  Support
SeqNum                                                                              
0               0  CLASS_1       65        6   0.915493  0.984848  0.948905       66
1               1  CLASS_2        1       42   0.976744  0.875000  0.923077       48


## <b><span style='color:#F1A424'>| 7.</span> Get transformed data with 'get_transformed_data'

In [28]:
trans_data=aml.get_transformed_data(cancer_df_test)

In [29]:
trans_data

{'rfe_test':          id  diagnosis  r_texture_mean  r_radius_worst  r_perimeter_worst  r_area_mean  r_radius_se  r_area_worst  r_concavity_mean  r_concave_points_mean  r_concavity_worst  r_texture_worst  r_concave_points_worst  r_area_se  r_perimeter_se  r_smoothness_worst  r_perimeter_mean  r_radius_mean
 0   8911230          0        0.183859        0.252215           0.232434     0.236432     0.182533      0.248271          0.006649               0.025801           0.006807         0.221178                0.043649   0.239211        0.168328            0.418779          0.322915       0.336897
 1    842302          1       -0.018172        1.030715           1.156910     0.801028     1.454033      1.668912          1.330230               1.150117           0.969231         0.161185                1.043649   3.408622        1.658168            0.759624          0.911198       0.852816
 2    868871          0        0.142704        0.235676           0.225192     0.225409     0.332147

In [31]:
trans_data['rfe_test']

id,diagnosis,r_texture_mean,r_radius_worst,r_perimeter_worst,r_area_mean,r_radius_se,r_area_worst,r_concavity_mean,r_concave_points_mean,r_concavity_worst,r_texture_worst,r_concave_points_worst,r_area_se,r_perimeter_se,r_smoothness_worst,r_perimeter_mean,r_radius_mean
869691,1,0.31320149652592183,0.34317779090372125,0.357961893266661,0.269500233535731,0.3044187425860024,0.3699490353112487,0.7353723404255319,0.5793588741204065,0.6132062627637849,0.48825442717744844,0.7333857648446716,0.41662016369047616,0.32019683536249105,0.5370892018779344,0.40595087071848684,0.37330544581299874
859575,1,0.5660074826296098,1.0,0.9956892835589275,0.9215319943951424,1.0,1.5296687295231162,0.4787234042553192,0.6215793588741203,0.36582709326072155,0.4954824719913263,0.7034998033818324,2.0751488095238093,1.0,0.35680751173708924,0.9204244031830238,0.9264079324502286
8610404,1,0.47728487439871714,0.6993502658003544,0.6758341236313476,0.6297991592713685,0.9386120996441283,0.9444848926101201,0.43306737588652483,0.5191555903049256,0.3851599727705922,0.42247921937116006,0.5977192292567832,1.6845238095238095,0.9030670682560556,0.6450704225352113,0.6955368469611347,0.7040824231156557
87880,1,0.6964190272581506,0.6656822209096279,0.673247693766704,0.42438113031293784,0.6678529062870701,0.8807790316709138,0.6906028368794326,0.7177482408131353,0.6325391422736555,1.0473436935309,0.7915847424302005,1.0676618303571428,0.6655071480083146,1.3267605633802815,0.5509168492676738,0.5290107676814626
89346,0,0.1966862640299305,0.10448907265209688,0.0904388309337012,0.09602989257356377,0.08926453143534996,0.09128139788860577,0.016400709219858155,0.027365129007036745,0.02001361470388019,0.26020961329960246,0.054659850570192674,0.06945219494047619,0.07903109489670385,0.16244131455399058,0.14496597854918694,0.15640250987683013
8711216,0,0.467129877071085,0.6077968103957472,0.6025519441331149,0.688183092013078,0.5404804270462634,0.7706589006188569,0.22828014184397163,0.2165754495699765,0.2562287270251872,0.5493314058547163,0.33189146677152964,0.925595238095238,0.574301107198914,0.06009389671361505,0.7451274362818592,0.763730730498102
911916,1,0.469802244788883,0.5587714116952156,0.6180705233209759,0.6280242877160205,0.2970047449584816,0.6866581725518748,0.9911347517730497,0.7185301016419077,0.80285908781484,0.3679074810263824,0.6979944946913094,0.611932663690476,0.4875493148941586,0.5295774647887324,0.7612732095490714,0.7180261832829808
86517,1,0.34206306787814006,0.8458357944477259,0.8189499094749547,0.8720224194301728,0.8873072360616847,1.2411721878412814,0.6458333333333334,0.6778733385457388,0.4454731109598366,0.4347668955547524,0.6582776248525363,1.945405505952381,0.8746447206549866,0.6478873239436619,0.8950524737631185,0.9047176388566116
879523,1,0.3185462319615178,0.5812167749556999,0.5801362186395379,0.5353573096683792,0.23235468564650064,0.7319803421914816,0.3346631205673759,0.3189992181391712,0.4529611980939414,0.2663534513913986,0.4923318914667715,0.4566127232142857,0.25507996436601194,0.6366197183098593,0.6341829085457271,0.6304903555658842
877159,1,0.5943345804382683,0.6987595983461312,0.678420553495991,0.8225128444652032,0.7737247924080666,0.9490353112486348,0.48891843971631205,0.4519155590304925,0.34513274336283184,0.42753885074087455,0.3609909555642941,1.6173270089285712,0.7509862978831715,0.06478873239436624,0.8489216929996541,0.8597877449841195


In [32]:
trans_data['pca_test']

id,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,diagnosis
902975,-0.3989718000809219,0.19267816175336372,-0.033496978347754,-0.29614422788189865,-0.4870890531070569,-0.2700160567066823,0.1961246597414868,0.09465196848387032,-0.09675141004109375,-0.15544541761497338,-0.12397183212540674,0.03213458408474876,-0.09399946944910381,-0.12992015212857916,0
879523,0.17169299520291137,-0.16627229765902884,-0.5120426247999309,-0.08885599342151565,-0.082031546434844,-0.003189515183349992,0.20396739318812246,0.15885196202304908,0.3276995637717368,0.017031120407365263,0.08526462135654028,0.05510136881970222,-0.059134663015759745,-0.0387181581751436,1
915186,0.3112156543693483,2.3615382601756254,-0.14807269571411014,-0.6415707656030618,0.09771732741807448,0.30692607547035367,0.016930303649085057,0.4717097234506095,-0.481246949949476,0.3316215120225472,0.22722783255321077,-0.3542349120495271,0.05222687675787655,0.10546445180067487,0
857438,0.015550736459711167,-0.5546455519972249,-0.14848700351785965,0.33652880823436354,-0.018322089954810772,0.2645226765278098,0.014667554337142272,0.009433947722337896,0.1254580790656177,0.006052170769410503,-0.04410986722593502,-0.06255671543354979,-0.10573788890224195,-0.00917609569878794,1
869691,0.6885505538134837,1.5264367159299148,0.023101213860021286,-0.22701912561073748,0.8277850582730129,-0.9319984247082114,1.016826367139113,-0.6865134378094421,-0.1826518364386398,0.03720487977743159,0.26253511425875,0.3440865095848753,-0.45212824950097774,0.03738847943898553,1
8911230,-1.2229668061718275,-0.002643047039146973,0.013031399889465112,-0.1985227047939505,0.443684535731524,-0.0064794177203146485,0.011154023427718845,-0.1193935245493675,0.027648487392054596,0.1297373685888109,-0.012448822519131597,0.0578833553321028,0.22941273061850215,-0.018857347463218516,0
924934,-0.5705934919952875,0.7412719614829613,0.7011675514179438,0.5517746964339709,-0.3601176451901976,0.05863261985715748,-0.5039067279327428,-0.251064493901173,0.09259103551070619,0.011500510980645917,-0.019637042749106937,-0.0582761970595191,-0.03743215529799527,-0.20228070810423848,0
89742801,1.3632801861113943,-0.34837827615185724,1.110866586866131,-0.06011736085356101,0.2878777007446769,1.0792307528067497,0.4807610325980999,0.1942936926871925,-0.1958158275579135,-0.12966054150079043,-0.08692278938013448,0.3265455130487856,-0.14742350511169733,-0.07619011968433523,1
872608,0.4099450232716748,2.4110235489322758,1.4738288409022144,-0.5048030420070233,-1.2415219191137576,-0.5121870486755565,0.20730036336747598,0.43388200320324355,-0.013562186766948828,0.23389049261823025,0.3651469885107508,0.026810528207727186,-0.3629649697855812,-0.5768620834177836,0
877159,0.8032668671950991,-0.5943648552856159,1.1937316408071146,-0.34456570764975186,-0.4597031724782159,0.2678304661077233,0.8077798063324324,0.055684089461062986,-0.017423596803952127,-0.13128562493712564,0.1458512381071237,0.007015079207279083,0.2399104582807084,-0.20243742585524738,1


## <b><span style='color:#F1A424'>| 8.</span> Remove deployed models

In [33]:
aml.remove_saved_models('cancer_models_top_3')

In [35]:
remove_context()

True